# 시맨틱 검색 - Semantic Search


## 시맨틱 검색 (Semantic Search)

오픈서치의 시맨틱 검색은 검색 쿼리의 의미를 이해하고, 그에 따라 가장 관련성 높은 결과를 사용자에게 제공하는 기능입니다. 이는 전통적인 키워드 기반 검색과 달리, 검색 쿼리의 맥락과 의미를 분석하여 보다 정확하고 관련성 높은 검색 결과를 제공합니다. 예를 들어 "아마존"이라는 단어가 상품명인지, 회사명인지, 아니면 지역명인지 등 쿼리의 의미를 파악하여 그에 맞는 검색 결과를 제공합니다.


## 신경망 검색(Neural Search)

수년 동안 고객들은 OpenSearch k-NN을 기반으로 시맨틱 검색 애플리케이션을 구축하기 위해서는 텍스트 임베딩 모델을 검색 및 수집 파이프라인에 통합하기 위해 추가적인 미들웨어를 구축해야 하는 부담이 있었습니다. 이제 Amazon SageMaker와 Amazon Bedrock와의 통합을 통해 신경망 검색을 강화하며 클러스터에서 실행되는 시맨틱 검색 파이프라인을 지원할 수 있습니다.

신경망 검색은 텍스트를 벡터로 변환하고 인덱싱 시간과 검색 시간 모두에서 벡터 검색을 용이하게 합니다. 인덱싱 중에 신경망 검색은 문서 텍스트를 벡터 임베딩으로 변환하고 텍스트와 그 벡터 임베딩을 모두 벡터 인덱스에 인덱싱합니다. 신경망 쿼리를 사용하는 경우 신경망 검색은 쿼리 텍스트를 벡터 임베딩으로 변환하고, 벡터 검색을 사용하여 쿼리와 문서 임베딩을 비교한 다음 가장 가까운 결과를 반환합니다.

문서를 인덱스에 인제스트하기 전에 문서는 기계 학습(ML) 모델을 통과하게 되며, 이 모델은 문서 필드에 대한 벡터 임베딩을 생성합니다. 검색 요청을 보내면 쿼리 텍스트나 이미지도 ML 모델을 통과하여 해당 벡터 임베딩을 생성합니다. 그런 다음 신경망 검색이 임베딩에 대한 벡터 검색을 수행하고 일치하는 문서를 반환합니다.

신경망 검색을 사용하면 OpenSearch API를 통해 인간의 언어로 검색 쿼리를 실행하고, Amazon SageMaker에서 호스팅되거나 Amazon Bedrock에서 관리하는 텍스트 임베딩을 통해 의미론적 이해와 유사성을 고려한 텍스트 임베딩을 사용하여 더 정확한 결과를 제공할 수 있습니다.


## 사전준비


필요한 패키지를 설치합니다.


In [ ]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3

키워드 검색 단계에서와 마찬가지로 데이터를 준비합니다.


In [ ]:
import pandas as pd
import requests

df = pd.read_csv("./data/movies.csv", low_memory=False)
df.head(5)

데이터의 스키마를 확인합니다.


In [ ]:
df.info()

이 워크샵에서 사용할 임베딩 모델인 Cohere Embed Multilingual의 Max Sequence 길이는 512입니다. 따라서 여기에 맞게 임베딩할 plot 컬럼의 최대 길이를 512로 truncate합니다.


In [ ]:
max_length = 509


def truncate_plot(plot):
    if len(plot) > max_length:
        return plot[:max_length] + "..."
    else:
        return plot


# Apply the function to the 'plot' column
df["plot"] = df["plot"].apply(truncate_plot)

줄거리의 길이가 512가 넘는 레코드가 있는지 확인합니다


In [ ]:
# Find all items in df  with plot value longer than 512 characters
def find_long_plot_items(df):
    long_plot_items = df[df["plot"].str.len() > 512]
    return long_plot_items


find_long_plot_items(df).count()

### OpenSearch 도메인에 연결


In [1]:
def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs

In [5]:
import boto3, json


region_name = "us-west-2"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]
aos_host

'search-opensearch-workshop-rgx5p7rgxyxhec6mi3kusldapu.us-west-2.es.amazonaws.com'

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

## 데이터 임베딩을 위한 Ingest Pipeline 생성


모델 배포 과정을 참고하여 배포된 모델의 ID를 확인하고 아래와 같이 변수에 초기화합니다.


In [ ]:
import requests

search_model = {"query": {"match": {"name": "OpenSearch-Cohere"}}, "size": 10}

response = requests.get(
    "https://" + aos_host + "/_plugins/_ml/models/_search", auth=auth, json=search_model
)
model_info = json.loads(response.text)
model_id = model_info["hits"]["hits"][0]["_id"]
model_id

## 인제스트 파이프라인 생성

영화의 줄거리 정보를 가진 특정 필드("plot")에 대한 벡터 임베딩을 생성하는 파이프라인을 설정합니다. 이러한 임베딩은 검색 인덱스 필드("vector_field")에 저장되며, 효율적인 유사성 검색 및 검색 작업에 사용될 수 있습니다.


In [ ]:
pipeline = {
    "description": "An neural search pipeline for movie index - OpenSearch-cohere-060124084807",
    "processors": [
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "plot": "vector_field",
                },
            }
        }
    ],
}

pipeline_id = "movie_plot_embedding_pipeline"
aos_client.ingest.put_pipeline(id=pipeline_id, body=pipeline)

## 인덱스 생성

movie_semantic 인덱스를 생성합니다. 아래 세팅 및 맵핑 정보 중 중요한 것은 다음과 같습니다.

-   index.knn: KNN 검색을 위해 True로 설정합니다.
-   default_pipeline: 위 단계에서 생성한 pipeline_id를 제공합니다.
-   index.knn.space_type: 임베딩 벡터끼리의 유사도를 파악할 때 사용할 알고리즘을 cosinesimil로 지정합니다


In [ ]:
index_name = "movie_semantic"

# aos_client.indices.delete(index=index_name)

movie_semantic = {
    "settings": {
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
        "index.knn": True,
        "default_pipeline": pipeline_id,
        "index.knn.space_type": "cosinesimil",
    },
    "mappings": {
        "properties": {
            "date": {
                "type": "float",
            },
            "genre": {
                "type": "text",
            },
            "main_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "plot": {
                "type": "text",
            },
            "rating": {"type": "float"},
            "supp_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "title": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vote_count": {"type": "long"},
            "year": {"type": "long"},
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },
        }
    },
}

aos_client.indices.create(index=index_name, body=movie_semantic)

## 데이터 인제스트

키워드 검색과 동일하게 데이터를 인제스트합니다. 위 단계에서 인덱스를 생성할 때 INGEST PIPELINE을 설정했기 때문에 직접 임베딩 모델을 호출하여 데이터를 벡터로 변환하지 않아도 됩니다. 단, 데이터가 인제스트될 때 임베딩 모델을 호출해야 하는 단계가 있기 때문에 parallel_bulk의 병렬도가 높으면 에러가 발생할 있습니다. 여기서는 `thread_count`와 `queue_size`를 각각 1로 낮춰줍니다. 이 단계가 완료되는데는 약 4~5분 정도 소요됩니다.


In [ ]:
from tqdm import tqdm
from opensearchpy import helpers

json_data = df.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line


def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=20, thread_count=1, queue_size=1
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

데이터 인제스트가 잘 마무리되었는지 확인합니다.


In [ ]:
# Refresh the index to make the changes visible
aos_client.indices.refresh(index=index_name)

count = aos_client.count(index=index_name)
print(count)

# 키워드 검색과 시맨틱 검색 결과 비교


## 키워드 검색을 위한 함수 생성

키워드 단계에서 정의한 키워드 검색 함수와 동일한 함수를 생성합니다.


In [ ]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["plot"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating"]
    )
    display(query_result_df)

## 시맨팀 검색을 위한 함수 생성

뉴럴 검색을 위한 함수를 생성합니다. 주목해야할 부분은 다음과 같습니다.

1. **`"_source": {"excludes": ["vector_field"]}`**: 검색 결과에서 **`vector_field`** 필드를 제외한 모든 필드를 반환합니다. 이는 벡터 데이터가 크기 때문에 전송 비용을 줄이기 위함입니다.
2. **`"query": { ... }`**: 실제 검색 쿼리를 정의합니다.
3. **`"neural": { ... }`**: 벡터 검색을 수행하기 위한 쿼리 유형입니다.
4. **`"vector_field": "vector_field_name"`**: 벡터 데이터가 저장된 필드 이름입니다.
5. **`"query_text": query_text`**: 검색할 텍스트 쿼리입니다.
6. **`"model_id": model_id`**: 벡터 임베딩을 생성하는 데 사용된 모델의 ID입니다.


In [ ]:
def semantic_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["vector_field"]},
        "query": {
            "neural": {"vector_field": {"query_text": query_text, "model_id": model_id, "k": 10}},
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating"]
    )
    display(query_result_df)

## 결과 비교

자연어 기반의 쿼리를 작성하고 키워드 검색과 시맨틱 검색의 결과를 비교해봅니다


In [ ]:
query_text = "지구의 영웅들이 힘을 합쳐 우주의 악당을 물리친다"

In [ ]:
keyword_search(query_text)

In [ ]:
semantic_search(query_text)
# print(index_name)